<a name="15"></a>

# Исследование надёжности заёмщиков банка

На основе предоставленных данных о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

Проверить следующие гипотезы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок;
- Есть ли зависимость между семейным положением и возвратом кредита в срок;
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок;
- Как разные цели кредита влияют на его возврат в срок.

Описание данных:
- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита.

# СОДЕРЖАНИЕ 
1. [Изучение предоставленной информации](#1)
2. [Предобработка данных](#2)<br>
2-1 [Обработка пропусков](#3)<br>
2-2 [Замена типа данных](#4)<br>
2-3 [Обработка дубликатов](#5)<br>
2-4 [Лемматизация](#6)<br>
2-5 [Категоризация данных](#7)<br>
3. [Проверка гипотез](#8)<br>
3-1 [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#9)<br>
3-2 [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#10)<br>
3-3 [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#11)<br>
3-4 [Как разные цели кредита влияют на его возврат в срок?](#12)<br>
4. [Общий вывод](#13)<br>
5. [Чек-лист готовности проекта](#14)<br>

<a name="1"></a>

##  Изучение предоставленной информации [ [вверх](#15)]

In [1]:
import pandas as pd

from pymystem3 import Mystem     
from collections import Counter

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
path = 'C:/users/v_kly'

In [3]:
data = pd.read_csv(path + '/datasets/data.csv')  

In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

**Вывод**

1. **Данные по количесву детей (children)** - по сводной информации выявлено отсутствие пропусков, и использование корректного типа данных. Необходимо проверить наличие возможных ошибок в виде отрицательных значений и выбросов.
2. **Данные по общему трудовому стажу в днях (days_employed)** - присутствуют пропуски в данных. Бросаются в глаза две особенности: отрицательные значения и чрезмерно огромные положительные значения, для заданного порядка(дни). Дробные значения возможно связаны с конвертацией рабочих часов/минут в дни. Тип данных при таком выводе верный, но после предобработки возможно понадобится изменить на int64.
3. **Возраст клиента в годах (dob_years)** - пропусков нет, тип данных верный, требутся проверка на отрицательные значения и выбросы.
4. **Уровень образования клиента (education)** - пропусков нет, тип данных верный. Замечено изменения регистра. 
5. **Идентификатор уровня образования (education_id)** - пропусков нет, тип данных верный. Необходимо выявить и проверить взаимосвязь индетификатора с данными об уровне образования, исключить ошибки.
6. **Семейное положение (family_status)** - пропусков нет, тип данных верный. Исключить выбросы, ошибки, проверить регистр, посмотреть уникальные значения.
7. **Идентификатор семейного положения (family_status_id)** - пропусков нет, тип данных верный. Необходимо выявить и проверить взаимосвязь индетификатора с данными о семейном положении, исключить ошибки.
8. **Пол клиента (gender)** - пропусков нет, тип данных верный. Проверить наличие ошибок в заполнении, посмотреть уникальные значения.
9. **Тип занятости (income_type)** - пропусков нет, тип данных верный. Проверить регистр, ошибки заполнения, уникальные значения.
10. **Имел ли задолженность по возврату кредитов (debt)** - пропусков нет, тип данных верный. Проверить ошибки заполнения.
11. **Ежемесячный доход (total_income)** - есть пропуски, тип данных верный. 
12. **Цель получения кредита (purpose)** - пропусков нет, тип данных верный. Необходима проверка уникальных значений, для поиска ошибок и сведения к единому виду данных.

<a name="2"> </a>

## Предобработка данных  [ [вверх](#15)]

<a name="3"> </a>

### Обработка пропусков [ [вверх](#15)]

Так как для текущего анализа, исходя из поставленных задач, важными критериями являются наличие детей, семейное положение, доход, цели кредита и информация о задолженности, обработаем столбцы: **"children", "family_status", "family_status_id", "total_income","purpose", "debt"**. <br>
Остальные столбцы исключим из датафрейма.

In [6]:
data_source = data #сохраним исходный вид датафрейма на всякий случай в переменной data_source

In [7]:
data = data.loc[: , ['children', 'family_status', 'family_status_id', 'total_income', 'purpose', 'debt']]
data.head()         # сформировали датафрейм с ключевыми столбцами

,children,family_status,family_status_id,total_income,purpose,debt
0,1,женат / замужем,0,253875.639453,покупка жилья,0
1,1,женат / замужем,0,112080.014102,приобретение автомобиля,0
2,0,женат / замужем,0,145885.952297,покупка жилья,0
3,3,женат / замужем,0,267628.550329,дополнительное образование,0
4,0,гражданский брак,1,158616.077870,сыграть свадьбу,0


#### Обрабатываем столбец **`children`**

In [8]:
data['children'].value_counts() # смотрим на уникальные значения данных о количестве детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

1. Попадается отрицательное значение "-1" в 47 случаях из 21525 (0.18%), возможно ошибка ввода и имелось ввиду значение "1", но возможно и автозаполнение пустых полей. В силу маленького веса в общем объеме данных, предлагаю исключить данных строки, так как количество детей одна из ключевых метрик в тех. задании по данному проекту, и нельзя допустить искажения данных, лучше незначительно уменьшить выборку.
2. Также присутствует статистический выброс со значение "20" в количестве 76 значений (0,35% из общей выборки). Допускаю, что значения могут носить достоверный характер, но при этом велика вероятность ошибочной кодировки или автозаполнения пропусков. При незначительном объеме значений к общей выборке, предлагаю также исключить данные строки.
3. Суммарное уменьшение выборки составит всего пол процента (0,52%).

In [9]:
data = data[(data['children'] != 20)&(data['children'] != -1)] # сформировали датафрем исключая вышеуказанные строки
     # контрольные значения по количеству строк были проверены, выкинуто было ровно столько строк, сколько и планировалось

#### Обрабатываем столбец **`family_status`** 

In [10]:
data['family_status'].value_counts() # Проверяем значения на корректность категорий

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [11]:
data['family_status'].isnull().value_counts() # Проверяем на наличие пропусков

False    21402
Name: family_status, dtype: int64

Данные в столбце `family_status` корректны для дальнейшего использования.

#### Обрабатываем столбец **`family_status_id`** 

In [12]:
data.groupby('family_status')['family_status_id'].value_counts() 
                                                # группируем, чтобы сверить корректность применения id  и отсутствие ошибок

family_status          family_status_id
Не женат / не замужем  4                    2799
в разводе              3                    1189
вдовец / вдова         2                     952
гражданский брак       1                    4160
женат / замужем        0                   12302
Name: family_status_id, dtype: int64

Данные в столбце `family_status_id` корректны для дальнейшего использования.

#### Обрабатываем столбец **`total_income`**

In [13]:
data['total_income'].isnull().value_counts() # выясняем количество пропусков 

False    19240
True      2162
Name: total_income, dtype: int64

Количество пропусков занимает 10% от общей выборки, доля значительная, но не критичная, так как важнее соблюсти в данном столбце истинные значения, а не усреднять, рискуя смазать аналитически важные показатели. Сократим выборку, удалив строки с пропусками. 

In [14]:
data.dropna(subset=['total_income'], inplace=True) #удаляем пропуски с перезаписью

In [15]:
data.reset_index(drop=True, inplace=True) # поправляем индексы, после удаления строк
data.info()     # смотрим на общую картину по столбцам на наличие пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19240 entries, 0 to 19239
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19240 non-null  int64  
 1   family_status     19240 non-null  object 
 2   family_status_id  19240 non-null  int64  
 3   total_income      19240 non-null  float64
 4   purpose           19240 non-null  object 
 5   debt              19240 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 902.0+ KB


**Вывод**

**Все необходимые для анализа столбцы с данными обработаны на наличие пропусков, общая выборка сократилась на 10%.** <br>
10% пропуском в данных связаны скорее всего с непредоставлением таковых данных от заемщиков, в силу либо отсутствия дохода, либо возможности его официального подтверждения. В таком случае, разброс по этим данных может быть очень значительным, и заполнять их какими либо значениями(например, медианой) считаю нецелесообразным, так как это одна из ключевых метрик в данном анализе.

<a name="4"></a>

###  Замена типа данных [ [вверх](#15)]

Прибегнем к изменению типа данных по столбцу total_income, чтобы облегчить восприятие, и избавиться от длинных дробных "хвостов". Используем для этого метод astype() с аргументом 'int'

In [16]:
data['total_income'] = data['total_income'].astype('int') # меняем тип данных с float64 на int64 с помощью метода astype()

In [17]:
data.head()

,children,family_status,family_status_id,total_income,purpose,debt
0,1,женат / замужем,0,253875,покупка жилья,0
1,1,женат / замужем,0,112080,приобретение автомобиля,0
2,0,женат / замужем,0,145885,покупка жилья,0
3,3,женат / замужем,0,267628,дополнительное образование,0
4,0,гражданский брак,1,158616,сыграть свадьбу,0


**Вывод**

Замена и перезапись столбца прошли успешно. По остальным использованным данным смена типа данных не требуется.

<a name="5"></a>

### Обработка дубликатов [ [вверх](#15)]

Поищем дубликаты в датафрейме с помощью метода duplicated()

In [18]:
data.duplicated().value_counts()  # проверяем на наличие дубликатов

False    19231
True         9
dtype: int64

Найдено 9 дубликатов в датафрейме. Применяя метод `drop_duplicates()` удалим их. И сбросим индексы методом `reset_index()`.

In [19]:
data = data.drop_duplicates().reset_index(drop=True)
                                    # избавляемся от дубликатов, и сбрасываем индексы, с перезаписью

In [20]:
data.head()

,children,family_status,family_status_id,total_income,purpose,debt
0,1,женат / замужем,0,253875,покупка жилья,0
1,1,женат / замужем,0,112080,приобретение автомобиля,0
2,0,женат / замужем,0,145885,покупка жилья,0
3,3,женат / замужем,0,267628,дополнительное образование,0
4,0,гражданский брак,1,158616,сыграть свадьбу,0


**Вывод**

Количество дубликатов оказалось очень незначительным, удалили данные строки. <br>
Возникновение дубликатов возможно вызвано неоднократной подачей заявок на кредит от заемщиков, таким образом данные по одним и тем же людям накапливаются, образуя дубликаты.

<a name="6"></a>

### Лемматизация     [ [вверх](#15)]

Поработаем со столбцом `purpose`

In [21]:
data['purpose'].value_counts()  # посмотрим на значения

свадьба                                   718
сыграть свадьбу                           688
на проведение свадьбы                     680
операции с недвижимостью                  612
покупка коммерческой недвижимости         595
покупка жилья                             590
покупка жилья для сдачи                   585
жилье                                     582
операции с коммерческой недвижимостью     576
строительство жилой недвижимости          576
операции с жильем                         575
покупка своего жилья                      573
недвижимость                              570
покупка жилья для семьи                   569
строительство недвижимости                561
операции со своей недвижимостью           555
строительство собственной недвижимости    553
покупка недвижимости                      547
покупка жилой недвижимости                543
ремонт жилью                              538
автомобиль                                452
на покупку своего автомобиля      

**Регистр везде однороден**, но количество значений таково, что глаз уже замыливается, и в случае с еще большим количеством значений, шансы на ошибку возрастают, поэтому проведем принудительное приведение к нижнему регистру в профилактических целях. Используем метод str.lower()

In [22]:
data['purpose'].str.lower() # приводим данные к нижнему регистру

0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
19226               операции с жильем
19227            сделка с автомобилем
19228                    недвижимость
19229    на покупку своего автомобиля
19230           на покупку автомобиля
Name: purpose, Length: 19231, dtype: object

**Необходимо провести лемматизацию, для выявления категориальных меток.** <br>
Выявив категориальные метки, сможем из них сформировать категории для группировки данных и дальнейшего анализа.

In [23]:
m = Mystem() # создание экземпляра класса

In [24]:
purpose = data['purpose'].unique() # смотрим уникальные значения по столбцу purpose
purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'получение образования',
       'сделка с подержанным автомобилем', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее образование',
       'покупка жилья для сдачи

In [25]:
lemmas_list=[]    #Создаем пустой список, для записи в него всех лемм из уникальных значений столбца purpose
for row in purpose:
    lemma = m.lemmatize(row)
    lemmas_list.extend(lemma)

In [26]:
Counter(lemmas_list) # посчитаем частотность и сформируем категории.

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'получение': 3,
         'сделка': 2,
         'подержанный': 1,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

Из списка выше **сформируем категориальные леммы: `недвижимость`, `автомобиль`, `образование`, `свадьба`, `жилье`.** 


In [27]:
# данная ячейка закомментирована, так как её выполнение занимает продолжительное время.
'''
def lemm_fun(purpose):      #создаем функцию для лемматизации столбца с целями кредита
    lemma = m.lemmatize(purpose)
    return lemma

data['lemmas'] = data['purpose'].apply(lemm_fun)  # методом apply применяем поочередно к каждой строке функцию лемматизации
'''

"\ndef lemm_fun(purpose):      #создаем функцию для лемматизации столбца с целями кредита\n    lemma = m.lemmatize(purpose)\n    return lemma\n\ndata['lemmas'] = data['purpose'].apply(lemm_fun)  # методом apply применяем поочередно к каждой строке функцию лемматизации\n"

Для удобства и скорости работы, процесс лемматизации был единожды проведен и данные записаны в соответствующий файл, с которым и будем работать в дальнейшем.

In [28]:
data = pd.read_csv(path + '/datasets/lemmatized_data.csv')

In [29]:
data['lemmas'].head()

0                 ['покупка', ' ', 'жилье', '\n']
1       ['приобретение', ' ', 'автомобиль', '\n']
2                 ['покупка', ' ', 'жилье', '\n']
3    ['дополнительный', ' ', 'образование', '\n']
4               ['сыграть', ' ', 'свадьба', '\n']
Name: lemmas, dtype: object

Необходимо создать функцию для сопоставления лемм и присваивания категориальных меток. <br>
Леммы `жилье` и `недвижимость` объединяем в одну категорию `недвижимость`, так как нет достоверных данных о видах недвижимости и соответственно мы не сможем детализировать данную категорию

In [30]:
def category_gen(lemm):           # функция сравнения лемм с присваиванием категории
    try:
        if 'недвижимость' in lemm:
            return 'недвижимость'
        elif 'жилье' in lemm:
            return 'недвижимость'
        elif 'автомобиль' in lemm:
            return 'автомобиль'
        elif 'образование' in lemm:
            return 'образование'
        elif 'свадьба' in lemm:
            return 'свадьба'
    except: 
        return 'неизвестно'
    
data['category'] = data['lemmas'].apply(category_gen) # записываем категории в новый столбец
data['category'].head()

0    недвижимость
1      автомобиль
2    недвижимость
3     образование
4         свадьба
Name: category, dtype: object

In [31]:
data['category'].value_counts() # посмотрим на распределение и результат разбивки по категориям

недвижимость    9700
автомобиль      3871
образование     3574
свадьба         2086
Name: category, dtype: int64

**Вывод**

Методом лемматизации удалось выявить 4 основных категории целей получения кредита и присвоить каждой строке данных определенную категорию, что позволит в дальнейшем группировать и анализировать группы по целям займа. <br><br>
Лемматизация и выделение категорий показали, что самой распространенной целью кредита является недвижимость. <br> <br>
Автомобиль и образование примерно равны, а на свадьбу кредит берут почти в 5 раз реже, чем на недвижимость. 

<a name="7"></a>

### Категоризация данных   [ [вверх](#15)]

####  Цели кредита

In [32]:
category = data.groupby('category').sum()            #группируем по категориям целей кредита 
category['count'] = data['category'].value_counts()  # вычисляем количество заявок на кредит по каждой категории
category

,Unnamed: 0,children,family_status_id,total_income,debt,ch_status,count
category,,,,,,,
автомобиль,37562027,1813,3792,647481118,364,1311,3871
недвижимость,93463841,4639,9394,1639280521,713,3325,9700
образование,34011516,1690,3450,587739173,330,1191,3574
свадьба,19868681,959,2086,346212066,155,702,2086


####  Наличие детей

In [33]:
children = data.groupby('children').sum()  #группируем по количеству детей 
children

,Unnamed: 0,family_status_id,total_income,debt,ch_status
children,,,,,
0,122521960,14249,2098031841,952,0
1,41318471,3517,743641422,408,4343
2,17749922,824,317996930,177,1850
3,2899462,115,53880175,22,294
4,306513,16,5799072,3,34
5,109737,1,1363438,0,8


Статистические выборки по каждой отдельной категории (количество детей) неравномерны, и не всегда достаточны для проведения дискретного анализа зависимости от точного количества детей в семье. <br>
Необходимо категорирование по фактору наличия или отсутствия детей, не взирая на количество.

In [34]:
def children_fun(children):  #функция для присваивания статуса по детям: есть/нет
    if children == 0:
        return 0
    else: return 1

                        #пробегаемся по колонке и применяем функцию, с последующим формированием новой категориальной колонки
data['ch_status'] = data['children'].apply(children_fun)
data['ch_status'].value_counts()         # считаем количество заявителей по каждой категории

0    12702
1     6529
Name: ch_status, dtype: int64

In [35]:
ch_status = data.groupby('ch_status').sum()   # считаем значение по задолженности для каждой категории
ch_status['debt']

ch_status
0    952
1    610
Name: debt, dtype: int64

Позже, сведем данные в одну таблицу, для проведения анализа.

#### Семейное положение

In [36]:
family = data.groupby('family_status_id').sum()  # группируем по семейному id статусу
family

,Unnamed: 0,children,total_income,debt,ch_status
family_status_id,,,,,
0,106423924,6270,1869915521,842,4329
1,35839134,1673,620287780,336,1254
2,7865574,132,123206473,56,100
3,10536942,462,183474119,75,365
4,24240491,564,423828985,253,481


Хотелось бы явного отображения семейного статуса в данной таблице, добавим столбец с расшифровкой id.

In [37]:
family['status'] = data.groupby('family_status_id')['family_status'].unique() # значение id записываем в новый столбец 'status'
family

,Unnamed: 0,children,total_income,debt,ch_status,status
family_status_id,,,,,,
0,106423924,6270,1869915521,842,4329,[женат / замужем]
1,35839134,1673,620287780,336,1254,[гражданский брак]
2,7865574,132,123206473,56,100,[вдовец / вдова]
3,10536942,462,183474119,75,365,[в разводе]
4,24240491,564,423828985,253,481,[Не женат / не замужем]


Для дальнейшего анализа необходим будет столбец с количеством заявок в каждой категории.

In [38]:
family['count'] = data.groupby('family_status_id')['family_status'].count() # количество заявок на каждый семейный статус
family

,Unnamed: 0,children,total_income,debt,ch_status,status,count
family_status_id,,,,,,,
0,106423924,6270,1869915521,842,4329,[женат / замужем],11065
1,35839134,1673,620287780,336,1254,[гражданский брак],3716
2,7865574,132,123206473,56,100,[вдовец / вдова],858
3,10536942,462,183474119,75,365,[в разводе],1078
4,24240491,564,423828985,253,481,[Не женат / не замужем],2514


Также, проведем еще одну категоризацию по принципу наличия либо отсутствия общего семейного бюджета у заявителя.<br> 
Уже имеющиеся категории выше **объединим в две категории** по логике человек один или в паре с кем-то. <br>
**Назовех их соответственно:** `couple` и `single`

In [39]:
def couple_st(status_id):   # функция фильтрации на две категории
    if status_id < 2:
        return 'couple'
    else: return 'single'
    
data['couple_st'] = data['family_status_id'].apply(couple_st)   # применение функции и запись в новый столбец

In [40]:
couple = data.groupby('couple_st').sum() # группируем по категориям couple и single

In [41]:
couple['count'] = data['couple_st'].value_counts() #считаем и добавляем в новый столбец количество заявок по каждой категории


In [42]:
couple[['debt', 'count']]  # выводим критичные для будущего анализа столбцы

,debt,count
couple_st,,
couple,1178,14781
single,384,4450


#### Уровень дохода

Для того, чтобы использовать данные о доходе для выявления зависимости с возникновением кредиторской задолженности, введем категориальное разделение всей выборки на процентные части от меньшего заработка к большему, чтобы иметь возможность проанализировать в дальнейшем возможное изменение ключевой метрики с ростом уровня дохода.

In [43]:
data['total_income'].quantile([.125, .25, .375, .5, .625, .75, .875]) # делим выборку дохода перцентилями с шагом в 12.5%

0.125     81278.50
0.250    103023.00
0.375    123919.75
0.500    145077.00
0.625    169667.50
0.750    203468.50
0.875    260938.25
Name: total_income, dtype: float64

In [44]:
def income_f(income):             # функция присвоения категории
    if income < 81278.50:
        return '0%-12.5%'
    elif income < 103023.00: 
        return '12.5%-25%'
    elif income < 123919.75:
        return '25%-37,5%'
    elif income < 145077.00:
        return '37,5%-50%'
    elif income < 169667.50:
        return '50%-62,5%'
    elif income < 203468.50:
        return '62,5%-75%'
    elif income < 260938.25:
        return '75%-87,5'
    else: return '87,5%-100%'

data['volume_income'] = data['total_income'].apply(income_f) # применение функции и запись в новый столбец

In [45]:
volume_income = data.groupby('volume_income').sum().sort_values(by='volume_income')
volume_income

,Unnamed: 0,children,family_status_id,total_income,debt,ch_status
volume_income,,,,,,
0%-12.5%,23165721,972,2193,154069466,182,698
12.5%-25%,23213828,1150,2428,222297850,201,803
"25%-37,5%",23254111,1105,2435,271985529,211,806
"37,5%-50%",23104256,1205,2456,323246569,206,872
"50%-62,5%",22856432,1120,2366,378468409,214,818
"62,5%-75%",22812026,1141,2286,446849808,209,826
"75%-87,5",23207724,1160,2317,552002011,173,834
"87,5%-100%",23291967,1248,2241,871793236,166,872


**Вывод**

Общие данные удалось категорировать в зависимости от поставленных задач и вопросов, были сформированы новые вспомогательные таблицы, которые будут использованы для анализа при ответе на ключевые вопросы проекта. 

<a name="8"></a>

## Проверка гипотез [ [вверх](#15)]

<a name="9"></a>

###  Есть ли зависимость между наличием детей и возвратом кредита в срок?  [ [вверх](#15)]

Для ответа на этот вопрос понадобиться категорированный ранее датафрейм `ch_status`, в который мы добавим столбец с количеством заявителей по каждой категории. 

In [46]:
ch_status['count'] = data['ch_status'].value_counts()#добавляем столбец с данными о количестве заявителей по каждой категории
ch_status

,Unnamed: 0,children,family_status_id,total_income,debt,count
ch_status,,,,,,
0,122521960,0,14249,2098031841,952,12702
1,62384105,9101,4473,1122681037,610,6529


Посчитаем процентное соотношение должников к количеству заявителей по каждой категории и выведем для наглядности только важные столбы.

In [47]:
ch_status['percent'] = ch_status['debt']/ch_status['count']*100
ch_status[['debt','count','percent']].round(1)

,debt,count,percent
ch_status,,,
0,952,12702,7.5
1,610,6529,9.3


Как мы видим из таблицы, процент наличия кредиторской задолженности в категории **"без детей" равен 7.5%**, а в категории **"с детьми" - 9,3%**. <br>
**Показатель возврата кредита в срок лучше в категории `без детей` на 1,8 процентных пункта.**


**Вывод**

**`Показатели каждой категории близки, чтобы отсекать при скоринге категорию "с детьми", но при наличии цели увеличения конверсионных показателей по денежному обороту и ограничении в количестве выдаваемых кредитов приоритезация в сторону категории "без детей" может дать прирост почти в 20% ((9.3-7.5)/9.3*100).`**

<a name="10"></a>

### Есть ли зависимость между семейным положением и возвратом кредита в срок?  [ [вверх](#15)]

Обратимся к первой категориальной таблице. Посчитаем и добавим новым столбцом отношение показателя задолженности к количеству заявителей по каждой категории. Выведем необходимые для анализа столбцы.

In [48]:
family['percent'] = family['debt']/family['count']*100

family[['status','debt','count','percent']].round(1)

,status,debt,count,percent
family_status_id,,,,
0,[женат / замужем],842,11065,7.6
1,[гражданский брак],336,3716,9.0
2,[вдовец / вдова],56,858,6.5
3,[в разводе],75,1078,7.0
4,[Не женат / не замужем],253,2514,10.1


Как видно из таблицы, самая ответственная категория по уплате кредита `[вдовец / вдова]` с показателем в 6.5%, а самая не надежная категория `[Не женат / не замужем]`, где каждый десятый имел наличие задолженности (10,1%). <br>

Теперь посмотрим вторую категориальную таблицу, где распределение ведется по наличию или отсутствию партнера, и как следствие предполагает наличие семейного бюджета.

In [49]:
couple['percent'] = couple['debt']/couple['count']*100
couple[['debt', 'count', 'percent']].round(1)

,debt,count,percent
couple_st,,,
couple,1178,14781,8.0
single,384,4450,8.6


По данным категориям очень назначительное расхождение - 0.6%. Семейные пары оказались, чуть-чуть более надежными заемщиками,чем одиночки. Таким показателем можно пренебречь, и не рассматривать данный вариант фильтра при скоринге.

**Вывод**

Существует достаточно серьезный разброс между наиболее "надежной" и "недобросоветсной" категориями - 3,6%. Так как превышение исчисляется не порядками, несколькими процентами, то отфильтровывать по одному фактору без понижающих дополнительных критериев большого смысла нет. При комплексном скоринге такая величина может уже сыграть роль, в случае набора дополнительных отрицательных метрик.

<a name="11"></a>

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?   [ [вверх](#15)]

Воспользуемся предварительной таблицей и добавим к ней два столбца: количество заемщиков на каждую категорию и процент людей имеющих факт кредитной задолженности. <br>
Выведем важные для анализа столбцы.

In [50]:
volume_income['count'] = data.groupby('volume_income')['total_income'].count()
volume_income['percent'] = volume_income['debt']/volume_income['count']*100
volume_income[['debt', 'count', 'percent']].round(1)

,debt,count,percent
volume_income,,,
0%-12.5%,182,2404,7.6
12.5%-25%,201,2404,8.4
"25%-37,5%",211,2404,8.8
"37,5%-50%",206,2403,8.6
"50%-62,5%",214,2404,8.9
"62,5%-75%",209,2404,8.7
"75%-87,5",173,2404,7.2
"87,5%-100%",166,2404,6.9


Деление выборки было равномерным с шагом в 12.5% на увеличение дохода, результаты в виде процента "ненадежных" заемщиков имеют крайне малый разброс и носят скорее случайный характер, нежели определенную закономерность. <br>

**Вывод**

Как мы видим, наличие бОльшего дохода не определяет надежность заемщика, так как главный критерий - это умение грамотно распоряжаться финансами, ведь с ростом доходов, растут и расходы. <br>
Можно лишь сделать предположения, глядя на самый лучший показатель в этой таблице (6,9%), что люди на вершине данной выборки по доходу не просто так туда попали, а обладают чуть более лучшими навыками управления личными финансами.

<a name="12"></a>

### Как разные цели кредита влияют на его возврат в срок?   [ [вверх](#15)]

Применим к сгруппированной таблице с целями кредита формулу расчета доли ненадежных заемщиков. Выведем важные столбцы.

In [51]:
category['percent'] = category['debt']/category['count']*100
category[['debt','count','percent']].round(1)

,debt,count,percent
category,,,
автомобиль,364,3871,9.4
недвижимость,713,9700,7.4
образование,330,3574,9.2
свадьба,155,2086,7.4


Как видно из таблицы наши категории поделились на два лагеря: более надежные с целями `свадьба` и `недвижимость`, и менее надежные с `автомобилем` и `образованием`. <br>
**Показатели практически идентичные: 7.4% - недвижимость и свадьба, 9.3% в среднем - автомобиль и образование.**

**Вывод**

Можно предположить следующее: <br>
1) бОльшая надежность в категории недвижимость скорее всего связана с грядущим долгосрочным обременением и как следствие более взвешенным и просчитанным решением. Тем более, в случае просрочек, есть риск потерять крышу над головой. <br>
2) Автомобиль и образование цели менее стоимостные в общем случае, и возможно более спонтанные в определенной доле от общего числа, что может приводить к более халатному отношению.<br>
3) Свадебная категория, хоть и близка по критериям к автомобилю и образованию, носит краткосрочный и средснесрочный характер, но компенсируется доброй традицией дарить деньги на данное событие, что возможно улучшает платежеспособность заемщика.<br>
<br>
Также, заметна разница в количестве обращений по разным целям: лидирует "недвижимость", обгоняя "автомобиль" и "образование" в 2-2.5 раза, и почти в 5 раз "свадьбу".

<a name="13"></a>

## Общий вывод   [ [вверх](#15)]

Проанализированные категории и зависимости в частном отдельном случае не дают повода отказывать в выдаче кредита, так как надежность заемщиков во всех рассмотренных категориях отличается на единицы процентов, а не на порядки. Но в случае действующего ограничения на момент времени по количеству выдаваемых кредитов, можно использовать проанализированные данные для выстраивания приоритизации и снижению рисков по совокупности категориальных факторов.

<a name="14"></a>